데이터 전처리

공통 패키지

In [1]:
#데이터 가져오기
import numpy as np
import pandas as pd
import seaborn as sns

#시각화 패키지
import matplotlib.pyplot as plt

#시각화할 때 한글을 출력하기 위해서
import platform
from matplotlib import font_manager, rc


from sklearn import preprocessing


#한글처리
#메켄도시의 경우
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = "c://Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)


수치 데이터

이상치 감지

In [2]:
#z-score를 이용하는 방법
#z-score:(데이터-평균)/표준편차가 절대값으로 3이 넘는 경우 이상치
#z-score를 이용한 이상치 감지를 위한 함수
def outliers_z_score(ys):
    #임계값 설정
    threshold = 3
    
    #평균 구하기
    mean_y = np.mean(ys)
    
    #표준편차 구하기
    stdev_y = np.std(ys)
    
    #z_score 구하기
    z_scores = [(y - mean_y) / stdev_y for y in ys]
    
    return np.where(np.abs(z_scores) > threshold)

features = np.array([[10, 10, 7, 6, 4, 5, 3, 3], [15000, 10, 7, 6, 4, 2, 2, 2]])
print(outliers_z_score(features))

(array([1], dtype=int64), array([0], dtype=int64))


In [3]:
#z-score를 보장 - MAD(중위 절대 편차)
def outliers_modified_z_score(ys):
    #임계값 설정
    threshold = 3.5
    
    #중앙값 구하기
    median_y = np.median(ys)
    
    #편차값 구하기
    median_absolute_deviation_y = np.median([np.abs(y-median_y) for y in ys])
    
    #보정한 z_score 구하기
    median_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y for y in ys]
   
    #이상치를 검출한 후 인덱스를 리턴
    return np.where(np.abs(median_z_scores) > threshold)

features = np.array([[10, 10, 7, 6, 4, 5, 3, 3], [15000, 10, 7, 6, 4, 2, 2, 2]])
print(outliers_modified_z_score(features))

(array([1], dtype=int64), array([0], dtype=int64))


In [4]:
#IQR을 이용하는 방법 - (3사분위수 - 1사분위수)
#1사분위 수에서 IQR*1.5 한 값을 뺀 것보다 적거나
#3사분위 수에서 IQR*1.5 한 값을 더한 것보다 큰 경우
def outliers_iqr(ys):
    #1사분위수와 3사분위수 구하기
    q1, q3 = np.percentile(ys, [25, 75])
    iqr = q3 - q1
    #경계값 계산
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    print("하한:", lower_bound)
    print("상한:", upper_bound)
    
    #이상치를 검출한 후 인덱스를 리턴
    return np.where((ys < lower_bound) | (ys > upper_bound))

features = np.array([[10, 10, 7, 6, 4, 5, 3, 3], [15000, 10, 7, 6, 4, 2, 2, 2]])
print(outliers_iqr(features))

하한: -4.125
상한: 14.875
(array([1], dtype=int64), array([0], dtype=int64))


In [5]:
#일정 비율을 이상치로 설정
from sklearn.covariance import EllipticEnvelope

#10퍼센트를 이상치로 간주하는 객체 생성
outlier_detector = EllipticEnvelope(contamination = 0.1)

#샘플 데이터 생성
from sklearn.datasets import make_blobs
features, _ = make_blobs(n_samples = 10,
                        n_features = 2,
                         centers = 1,
                         random_state = 42)
print(features)

#1번 행의 데이터를 이상치로 변경
features[1, 0] = 2000
features[1, 1] = 1500
print(features)

features = np.array([[10, 10, 7, 6, 4, 5, 3, 3], [15000, 10, 7, 6, 4, 2, 2, 2]])
#훈련
outlier_detector.fit(features)
#예측
outlier_detector.predict(features)

[[-2.743351    8.78014917]
 [-3.4172217   7.60198243]
 [-3.52202874  9.32853346]
 [-2.26723535  7.10100588]
 [-2.97261532  8.54855637]
 [-1.04354885  8.78850983]
 [-1.86150908 10.53731598]
 [-2.97867201  9.55684617]
 [-4.23411546  8.4519986 ]
 [-0.92998481  9.78172086]]
[[-2.74335100e+00  8.78014917e+00]
 [ 2.00000000e+03  1.50000000e+03]
 [-3.52202874e+00  9.32853346e+00]
 [-2.26723535e+00  7.10100588e+00]
 [-2.97261532e+00  8.54855637e+00]
 [-1.04354885e+00  8.78850983e+00]
 [-1.86150908e+00  1.05373160e+01]
 [-2.97867201e+00  9.55684617e+00]
 [-4.23411546e+00  8.45199860e+00]
 [-9.29984808e-01  9.78172086e+00]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


array([1, 1])

In [9]:
houses = pd.DataFrame()
houses['Price'] = [534433, 392333, 293222, 4322032]
houses['Bathrooms'] = [2, 3.5, 2, 116]
houses['Square_Feet'] = [1500, 2500, 1500, 48000]
print(houses)
print()

#새로운 특성 추가 - bathrooms가 20이 넘는 경우 새로운 특성으로 변환
# 불리언 조건을 기반으로 특성을 만듭니다.
houses["Outlier"] = np.where(houses["Bathrooms"] < 20, 0, 1)
# 데이터를 확인합니다.
print(houses)
print()

#특성 변환 - 로그 특성
houses["Log_Of_Square_Feet"] = [np.log(x) for x in houses["Square_Feet"]]
# 데이터를 확인합니다.
print(houses)
print()

#스케일링
from sklearn import preprocessing

#이상치에 덜 민감한 스케일링
scaler = sklearn.preprocessing.RobustScaler()
scaler.fit(houses[['bathrooms']])
print(scaler.transform(houses[['bathrooms']]))

#이상치에 민감한 스케일링
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(houses[['bathrooms']])
print(scaler.transform(houses[['bathrooms']]))

scaler = sklearn.preprocessing.MinMaxScaler()
scaler.fit(houses[['bathrooms']])
print(scaler.transform(houses[['bathrooms']]))

     Price  Bathrooms  Square_Feet
0   534433        2.0         1500
1   392333        3.5         2500
2   293222        2.0         1500
3  4322032      116.0        48000

     Price  Bathrooms  Square_Feet  Outlier
0   534433        2.0         1500        0
1   392333        3.5         2500        0
2   293222        2.0         1500        0
3  4322032      116.0        48000        1

     Price  Bathrooms  Square_Feet  Outlier  Log_Of_Square_Feet
0   534433        2.0         1500        0            7.313220
1   392333        3.5         2500        0            7.824046
2   293222        2.0         1500        0            7.313220
3  4322032      116.0        48000        1           10.778956



NameError: name 'sklearn' is not defined

결측치 확인

In [14]:
#seaborn의 titanic 데이터 가져오기
titanic = sns.load_dataset('titanic')

#RangeIndex의 개수와 열의 데이터 개수가 맞지 않으면 결측치가 존재
titanic.info()

#각 데이터 개수를 리턴 - 옵션이 없으면 None은 제외
print(titanic['deck'].value_counts())

#None을 제외하지 않고 리턴
print(titanic['deck'].value_counts(dropna = False))

#isnull 이용
print(titanic['deck'].isnull().sum(axis=0))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int6

결측치 처리

In [25]:
#결측치 제거
import numpy as np
features = np.array([
    [1.1, 11.1],[2.2, 22.2],[3.3, 33.3],[4.4, 44.4],[np.nan, 55.5]])
print(features)
print()
#(~ 연산자를 사용하여) None 제거
#isnan이 True인 데이터를 제외하고 가져오기
print(features[~np.isnan(features).any(axis=1)])
print()

titanic.info()
#컬럼별 null 데이터의 개수 찾기
print(titanic.isnull().sum(axis=0))
print()
#NaN 값이 500개 이상인 컬럼 제거
titanic_thresh = titanic.dropna(axis=1, thresh=500)
print(titanic_thresh.columns)
titanic_thresh.info()

#age가 None인 데이터 삭제
titanic_age=titanic_thresh.dropna(subset=['age'], how='any')
titanic_age.info()


[[ 1.1 11.1]
 [ 2.2 22.2]
 [ 3.3 33.3]
 [ 4.4 44.4]
 [ nan 55.5]]

[[ 1.1 11.1]
 [ 2.2 22.2]
 [ 3.3 33.3]
 [ 4.4 44.4]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), i

결측치 대체

In [30]:
#829번째 데이터에 NaN이 있음
print(titanic['embark_town'][825:835])
print()

#이전 값으로 
#ffill대신 bfill 가능
#method를 제거하고 다른 값으로 작성해도 됨
titanic_fill_ffill = titanic['embark_town'].fillna(method='ffill', inplace=True)
print(titanic['embark_town'][825:835])

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
830      Cherbourg
831    Southampton
832      Cherbourg
833    Southampton
834    Southampton
Name: embark_town, dtype: object

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829     Queenstown
830      Cherbourg
831    Southampton
832      Cherbourg
833    Southampton
834    Southampton
Name: embark_town, dtype: object


In [33]:
#가장 자주 등장하는 데이터로 채우기
titanic = sns.load_dataset('titanic')
#embark_town의 값의 개수를 확인
mode = titanic['embark_town'].value_counts()
print(mode)
print()

#결측치를 가장 빈번히 등장하는 단어로 채우기
titanic['embark_town'].fillna(mode.idxmax(), inplace=True)
print(titanic['embark_town'][825:835])

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
830      Cherbourg
831    Southampton
832      Cherbourg
833    Southampton
834    Southampton
Name: embark_town, dtype: object


함수형 데이터 - category

one hot encoding

In [39]:
mpg = pd.read_csv('c://programming/python/pandasdata/auto-mpg.csv', header=None)

#열이름 설정
mpg.columns = ['mpg', 'cylinders', 'displacement', 'horsepower','weight', 
               'acceleration', 'model year', 'origin', 'name']

mpg.info()

#horsepower를 3개 구간으로 분할

#실수로 변환하기 위해서 ?를 None으로 치환
mpg['horsepower'].replace('?', np.nan, inplace=True)

#horsepower가 None인 행을 삭제
mpg.dropna(subset=['horsepower'], axis=0, inplace= True)

#실수로 변환
mpg['horsepower'] = mpg['horsepower'].astype('float')
mpg.info()


#구간의 이름을 생성
bin_names=['저출력', '보통출력', '고출력']
#구간의 경계값을 생성
count, bin_dividers = np.histogram(mpg['horsepower'], bins=3)
print(bin_dividers)
print()

#구간 분할
mpg['hp_bin'] = pd.cut(x=mpg['horsepower'],
                      bins=bin_dividers,
                      labels=bin_names,
                      include_lowest=True)

print(mpg)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
dtypes: float64(4), int64(3), object(2)
memory usage: 28.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float6

In [40]:
#onehotencoding
#범주형 데이터의 개수만큼 컬럼이 생성되고 자신의 값에 해당하는 컬럼에만 1을 대입하고 나머지는 0
horsepower_dummies = pd.get_dummies(mpg['hp_bin'])
print(horsepower_dummies)


     저출력  보통출력  고출력
0      0     1    0
1      0     1    0
2      0     1    0
3      0     1    0
4      0     1    0
..   ...   ...  ...
393    1     0    0
394    1     0    0
395    1     0    0
396    1     0    0
397    1     0    0

[392 rows x 3 columns]


In [44]:
from sklearn.preprocessing import LabelBinarizer
# 원-핫 인코더 생성
one_hot = preprocessing.LabelBinarizer()
# 특성을 원-핫 인코딩
print(one_hot.fit_transform(mpg['hp_bin'])

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_18104/2005028362.py, line 5)

In [45]:
#여러 개의 특성을 원 핫 인코딩
multiclass_feature = [("Python", "Java"),
                     ("C++", "Python"),
                     ("C++", "Java"),
                     ("Java", "Javascript"),
                     ("Python", "Javascript")]

one_hot_multiclass = preprocessing.MultiLabelBinarizer()
print(one_hot_multiclass.fit_transform(multiclass_feature))
print(one_hot_multiclass.classes_)

[[0 1 0 1]
 [1 0 0 1]
 [1 1 0 0]
 [0 1 1 0]
 [0 0 1 1]]
['C++' 'Java' 'Javascript' 'Python']


In [46]:
#하나의 컬럼에 일련번호 형태로 원 핫 인코딩
label_encoder = preprocessing.LabelEncoder()
print(label_encoder.fit_transform(mpg['hp_bin']))

[1 1 1 1 1 0 0 0 0 0 0 1 1 0 2 2 2 2 2 2 2 2 2 1 2 0 0 0 0 2 2 2 2 2 2 2 2
 1 0 1 1 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 0 1 1 1 0 1 1 0 2 1 1 1
 1 1 2 2 2 2 2 2 2 2 0 1 1 1 1 0 1 1 1 0 0 0 2 2 2 2 2 2 1 1 0 0 2 2 2 2 2
 2 2 2 1 0 2 2 2 1 1 1 1 0 2 2 2 2 2 2 2 2 1 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 1 1 1 1 2 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 1 1 0 1 1 1 2 2 2 2 2 1 1 1
 1 1 2 2 2 0 0 0 1 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 1 1 1 2 2 1 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 1 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2
 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2]


In [47]:
#희소행렬로 표현 - 0이 아주 많은 행렬이라서 0이 아닌 좌표만 가지고
features = [["Java", 1], ["C++", 2], ["C#", 1], ["Python", 2]]

onehot_encoder = preprocessing.OneHotEncoder()
print(onehot_encoder.fit_transform(features))

#밀집행렬로 표현 - 원 핫 인코딩과 유사한 결과
onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
print(onehot_encoder.fit_transform(features))

  (0, 2)	1.0
  (0, 4)	1.0
  (1, 1)	1.0
  (1, 5)	1.0
  (2, 0)	1.0
  (2, 4)	1.0
  (3, 3)	1.0
  (3, 5)	1.0
[[0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]]


순서가 있는 경우의 변환

In [52]:
df = pd.DataFrame({"점수":["저조", "보통", "우수", "보통", "저조"]})
print(df)
print()

scale_mapper = {"저조":1, "보통":2, "우수":3}

df['encoder'] = df['점수'].replace(scale_mapper)
print(df)

   점수
0  저조
1  보통
2  우수
3  보통
4  저조

   점수  encoder
0  저조        1
1  보통        2
2  우수        3
3  보통        2
4  저조        1


In [54]:
#문자열이나 숫자(문자로 취급)를 정렬해서 인코딩
features = np.array([["Low", 10],
                    ["High", 3],
                    ["Medium", 27]])

ordinal_encoder = preprocessing.OrdinalEncoder()
print(ordinal_encoder.fit_transform(features))
#알파벳 순서, 숫자는 크기가 아닌 맨 앞자리의 숫자 순서

[[1. 0.]
 [0. 2.]
 [2. 1.]]


Dictionary를 특성 행렬로 변환

In [59]:
# 라이브러리를 임포트합니다.
from sklearn.feature_extraction import DictVectorizer

# 딕셔너리를 만듭니다.
data_dict = [{"Red": 2, "Blue": 4},
             {"Red": 4, "Blue": 3},
             {"Red": 1, "Yellow": 2},
             {"Red": 2, "Yellow": 2}]

# DictVectorizer 객체를 만듭니다.
dictVectorizer = DictVectorizer(sparse=False)

# 딕셔너리를 특성 행렬로 변환합니다.
features = dictVectorizer.fit_transform(data_dict)
print(features)
print()

# 특성 이름을 얻습니다.
feature_names = dictVectorizer.get_feature_names()
print(feature_names)
print()

# 특성으로 데이터프레임을 만듭니다.
pd.DataFrame(features, columns=feature_names)

[[4. 2. 0.]
 [3. 4. 0.]
 [0. 1. 2.]
 [0. 2. 2.]]

['Blue', 'Red', 'Yellow']



,Blue,Red,Yellow
0,4.0,2.0,0.0
1,3.0,4.0,0.0
2,0.0,1.0,2.0
3,0.0,2.0,2.0


결측치 대체

머신러닝 이용 권장하나, 
데이터가 많은 경우 알고리즘 수행시간이 많이 소요되므로 자주 등장하는 값으로 대체하는 경우 존재

In [60]:
from sklearn.neighbors import KNeighborsClassifier

#두 가지의 행렬 생성
#X와 같은 대문자의 값은 훈련 데이터

# 범주형 특성을 가진 특성 행렬 생성
X = np.array([[0, 2.10, 1.45],
              [1, 1.18, 1.33],
              [0, 1.22, 1.27],
              [1, -0.21, -1.19]])

# 범주형 특성에 누락된 값이 있는 특성 행렬 생성
X_with_nan = np.array([[np.nan, 0.87, 1.31],
                       [np.nan, -0.67, -0.22]])

# KNN 학습기를 훈련
clf = KNeighborsClassifier(3, weights='distance')
#1번열 이후의 데이터를 가지고 0번 열을 예측하는 모델을 생성
trained_model = clf.fit(X[:,1:], X[:,0])
# 누락된 값의 클래스를 예측
imputed_values = trained_model.predict(X_with_nan[:,1:])
print(imputed_values)
print()


# 예측된 클래스와 원본 특성을 열로 합침
X_with_imputed = np.hstack((imputed_values.reshape(-1,1), X_with_nan[:,1:]))

# 두 특성 행렬을 연결
np.vstack((X_with_imputed, X))

[0. 1.]



array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

In [61]:
# 두 개의 특성 행렬을 합침
X_complete = np.vstack((X_with_nan, X))

#가장 자주 등장하는 데이터로 누락된 클래스 대체
imputer = Imputer(strategy='most_frequent', axis=0)
imputer.fit_transform(X_complete)


# 두 개의 특성 행렬을 합침
X_complete = np.vstack((X_with_nan, X))
#가장 자주 등장하는 데이터로 누락된 클래스 대체
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

NameError: name 'Imputer' is not defined

불균형한 데이터 다루기

In [66]:
from sklearn.ensemble import RandomForestClassifier

#분류기를 만들 때 가중치를 적용
weights = {0:.9, 1:.1}
rfc = RandomForestClassifier(class_weight=weights)
print(rfc)
print()


# 균형잡힌 클래스 가중치 적용
rfc = RandomForestClassifier(class_weight="balanced")
print(rfc)

RandomForestClassifier(class_weight={0: 0.9, 1: 0.1})

RandomForestClassifier(class_weight='balanced')


In [70]:
#0이 10개이고 1이 100개인 불균형한 데이터 만들기
list1 = []
list2 = []
for i in range(0,10,1):
    list1.append(0)
for i in range(0,100,1):
    list2.append(1)
target = np.array(list1+list2)
print(target)


[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [77]:
#0번과 1번의 인덱스를 추출
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
print(i_class0)
print(i_class1)

#데이터의 개수 확인
n_class0 = len(i_class0)
n_class1 = len(i_class1)

#다운 샘플링 - 랜덤하게 추출
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace = False)
print(i_class1_downsampled)
print()
np.hstack((target[i_class0], target[i_class1_downsampled]))

#업 샘플링
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace = True)
np.hstack((target[i_class0_upsampled], target[i_class1]))

[0 1 2 3 4 5 6 7 8 9]
[ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 100 101 102 103 104 105 106 107 108 109]
[ 85  46  97  10  68  37  36  91 107  27]



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

텍스트 마이닝

파이썬의 str클래스에서 제공하는 함수를 이용한 가공

In [90]:
text_data = ['My name is Hyeryun.',
            '     .I am a programmer',
            'We are the one.      ']
print(text_data)
print()

#좌우공백 제거
strip_text = [string.strip() for string in text_data]
print(strip_text)
print()

#모두 대문자로 만들기
upper_text = [string.upper() for string in text_data]
print(upper_text)
print()

#모두 대문자로 만들기 - 함수를 이용
def capitalizer(string:str) -> str:
    return string.upper()
print([capitalizer(string) for string in text_data])
print()

#모두 소문자로 만들기
lower_text = [string.lower() for string in text_data]
print(lower_text)
print()

#마침표 제거
remove_periods = [string.replace('.', '') for string in text_data]
print(remove_periods)
print()

#모든 공백 제거
remove_periods = [string.replace(' ', '') for string in text_data]
print(remove_periods)
print()

['My name is Hyeryun.', '     .I am a programmer', 'We are the one.      ']

['My name is Hyeryun.', '.I am a programmer', 'We are the one.']

['MY NAME IS HYERYUN.', '     .I AM A PROGRAMMER', 'WE ARE THE ONE.      ']

['MY NAME IS HYERYUN.', '     .I AM A PROGRAMMER', 'WE ARE THE ONE.      ']

['my name is hyeryun.', '     .i am a programmer', 'we are the one.      ']

['My name is Hyeryun', '     I am a programmer', 'We are the one      ']

['MynameisHyeryun.', '.Iamaprogrammer', 'Wearetheone.']



텍스트 데이터 전처리

In [ ]:
정규표현식

1)횟수 관련 메타 문자
*: 0회 이상의 반복

+: 1회 이상의 반복

?" 0 번이나 1회

{정수}: 정수 만큼 반복

{정수1, 정수2}: 정수1 에서 정수2만큼 반복

{정수, }: 정수 이상 반복


a{3, 5}: a를 3번에서 5번
a{3,}: a를 3번 이상

2)매칭 관련 메타 문자
.: 줄바꿈을 제외한 1개 문자

^: 시작을 나타내는 기호, [ ] 안에서 사용되면 제외
    ^a - a로 시작하는 , [^a] - a를 제외한

$: 끝을 나타내는 기호, [ ]안에서 사용되면 $

[ ]: 문자 집합으로 ,를 이용해서 구분할 수 있고 -를 이용해서 범위를 설정할 수 있습니다.
     [ac] - a 또는 c 
     [a-c] - a에서 c까지

|: 또는	

( ): 정규식을 그룹으로 묶기	


3)특수 문자
\\: \

\d: 숫자

\D: 숫자를 제외한

\s: 화이트 스페이스(공백이나 엔터와 같은 제어문자)

\S: 화이트 스페이스를 제외한

\w: 글자(숫자 나 문자)

\W: 글자를 제외한

\b: 단어의 경계

\B: 단어의 경계를 제외한

4)플래그
re.I: 대소문자 구분하지 않음

re.M: 여러 줄에 걸쳐서 찾음

5)정규식은 문자열 클래스 와 정규식 관련 클래스를 통해 사용
=>문자열 클래스의 메서드를 확인할 때 regexp 나 reg 또는 pattern 이라는 단어가 들어가면 정규 표현식을 의미합니다.

6)re 모듈의 함수
=>compile(pattern[, flags]): 정규식 객체를 생성
=>search
=>match
=>split
=>findall
=>sub: 치환

7)Match 인스턴스의 함수
=>group(): 찾은 문자열을 반환
=>groups(): 매칭된 전체 그룹 문자열을 튜플로 반환
=>start(): 매칭된 문자열의 시작 위치를 리턴
=>end(): 매칭된 문자열의 끝 위치를 리턴
=>span: 시작 위치 와 끝 위치를 튜플로 리턴

In [3]:
import re

# 특수 문자 와 숫자 제거
string = "★안녕하세요 반갑습니다. 112312321"

#특수 문자 제거
p = re.compile("\W+") #단어가 아닌 - 숫자 나 문자가 아닌
print(p)
result = p.sub(" ", string)
print(result)

#숫자 제거
p = re.compile("\d+") #숫자
result = p.sub(" ", result)
print(p)
print(result)

re.compile('\\W+')
 안녕하세요 반갑습니다 112312321
re.compile('\\d+')
 안녕하세요 반갑습니다  


텍스트 전처리 패키지

1)nltk 패키지
=>자연어 처리 와 관련된 여러 알고리즘을 구현한 패키지 - 자연어 처리에서는 필수
=>한 개의 패키지로 구성되어 있지 않고 여러 개의 패키지로 구성되어 있습니다.
=>설치
pip install nltk

상황에 따라서 pip를 업그레이드 하라는 메시지가 출력되기도 하는데 이 경우는 메시지 대로 명령어를 수행하면 됩니다.
 
2)nltk 안의 punkt
=>토큰화를 위한 패키지
=>단어 단위나 문장 단위로 분할해주는 기능을 토큰화라고 합니다.
=>파이썬 코드로 설치
import nltk
nltk.download('punkt')

3)nltk 의 stopwords(불용어)
=>불용어를 위한 패키지
import nltk
nltk.download('stopwords')


4)토큰화
=>단어 단위나 문장 단위로 분할하는 작업
=>nltk 패키지의 tokenize 의 word_tokenize 나 sent_tokenize 함수를 이용

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


In [4]:
#토큰화 패키지 - 단어, 문장단위 분할
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

#불용어
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\90x61\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\90x61\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

토큰화
=>단어 단위나 문장 단위로 분할하는 작업
=>nltk 패키지의 tokenize 의 word_tokenize 나 sent_tokenize 함수를 이용

In [5]:
#문장 토큰화
string = "I am a boy. You are a girl."
#구두점을 기준으로 분할해서 list로 리턴
print(sent_tokenize(string))

#단어 토큰화 - 공백을 기준으로 분할해서 list로 리턴
print(word_tokenize(string))


['I am a boy.', 'You are a girl.']
['I', 'am', 'a', 'boy', '.', 'You', 'are', 'a', 'girl', '.']


불용어
=>텍스트 데이터에서 특별한 의미를 부여하기 힘든 단어
=>영어의 경우는 nltk 패키지 와 sklearn 에서 불용어 리스트를 제공
=>nltk 의 경우는 다른 주요 언어(유럽)도 제공
=>한국어는 제공되지 않기 때문에 직접 작성
=>한국어 불용어 리스트를 제공해주는 사이트
https://www.ranks.nl/stopwords/korean
https://github.com/6/stopwords-json/blob/master/dist/ko.json
https://github.com/stopwords-iso/stopwords-ko/blob/master/stopwords-ko.txt

corpus
=>분석 대상이 되는 글자의 단위

불용어(stopword - 의미없는 단어) 제거

In [6]:
from nltk.corpus import stopwords

#영어 불용어 처리
word_english = ['I', "am", "a", "boy", "and", "you", "movie"]

#word_english 의 모든 내용을 w에 대입하고
#w 가 stopwords.word('english')에 속하지 않은 경우만 가지고
#list를 생성
result = [w for w in word_english 
              if not w in stopwords.words('english')]
print(result)
print()

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

result = [w for w in word_english 
              if not w in ENGLISH_STOP_WORDS]
print(result)
print()


#한글 불용어 처리
sentence = "나는 이번 주말에 서점에 가서 강화학습 책을 볼 예정입니다." 

#불용어 사전
stopwords_kor = ['나는', '가서']

from nltk.tokenize import word_tokenize
#문장을 단어 단위로 토큰화
words_kor = word_tokenize(sentence)

result = [i for i in words_kor if i not in stopwords_kor]
print(result)

['I', 'boy', 'movie']

['I', 'boy', 'movie']

['이번', '주말에', '서점에', '강화학습', '책을', '볼', '예정입니다', '.']


7)어간 추출
=>스펠이 다르더라도 의미가 같은 단어를 추출하는 것을 어간 추출
=>한국어는 어미, 조사에 따라 단어의 형태가 바뀌고 영어의 경우 주어의 형태, 시제에 따라 동사의 형태가 바뀌고 단수 냐 복수 냐에 따라 s 나 es 가 붙음
=>어간 추출을 하지 않고 텍스트 분석을 같은 의미를 갖는 단어인데 스펠이 달라서 다른 단어로 인식을 하게 되면 잘못된 분석 결과를 만들 수 있습니다.
=>영문의 경우는 NLTK 패키지의 PorterStemmer 나 LancasterStemmer 라이브러리를 이용해서 할 수 있고 한국어는 konlpy 와 같은 형태소 분석기를 이용

In [8]:

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

string = 'All pythoners have pythoned at least once'
print(string)

#단어 단위로 토큰화
words = word_tokenize(string)
print(words)

#어간 추출
ps_stemmer = PorterStemmer()
for w in words:
    print(ps_stemmer.stem(w), end=' ')
print()


from nltk.stem import LancasterStemmer
ls_stemmer = LancasterStemmer()
for w in words:
    print(ls_stemmer.stem(w), end=' ')
print()

All pythoners have pythoned at least once
['All', 'pythoners', 'have', 'pythoned', 'at', 'least', 'once']
all python have python at least onc 
al python hav python at least ont 


n-gram
=>n 번 이상 연이어 등장하는 단어들의 연속적인 모임
=>2개 연속해서 등장하면 바이그램, 3개 연속해서 등장하면 트라이그램
=>2개 이상의 단어의 조합이 하나의 의미를 갖는 경우 이를 하나로 처리
United Kingdom: 2개를 묶어서 하나의 단어로 판단
United States America, Republic Of Korea: 3개를 묶어서 하나의 단어로 판단

=>nltk 패키지의 ngrams 를 이용하면 생성할 수 있음

In [9]:
from nltk import ngrams
from nltk.tokenize import word_tokenize

sentence = "대한 민국 우리 나라 대한 민국"

#2개씩 묶기 - bigram
grams = ngrams(word_tokenize(sentence), 2)
for i in grams:
    print(i, end=' ')
print()

#3개씩 묶기 - trigram
grams = ngrams(word_tokenize(sentence), 3)
for i in grams:
    print(i, end=' ')

('대한', '민국') ('민국', '우리') ('우리', '나라') ('나라', '대한') ('대한', '민국') 
('대한', '민국', '우리') ('민국', '우리', '나라') ('우리', '나라', '대한') ('나라', '대한', '민국') 

영문 형태소 분석 - 품사 태깅, 문장을 의미를 갖는 corpus 로 분할하고 품사를 할당
=>영문의 경우는 nltk 패키지의 averaged_perceptron_tagger 를 설치
=>함수에 단어의 list를 대입하면 단어 와 품사 태그로 이루어진 튜플의 리스트를 리턴
=>품사 태그
NNP: 고유명사 등의 형태로 이미 설정되어 있음 

In [12]:
#영문 형태소 분석을 위한 라이브러리 설치
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk import pos_tag
from nltk import word_tokenize

sentence = 'I am a boy You are a girl'

tokens = word_tokenize(sentence)
print(tokens)

#품사 태깅
tags = pos_tag(tokens)
print(tags)

## 특정 품사만 골라내기
print([word for word, tag in tags 
       if tag in ['NN', 'NNS', 'NNP', 'NNPS']])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\90x61\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


['I', 'am', 'a', 'boy', 'You', 'are', 'a', 'girl']
[('I', 'PRP'), ('am', 'VBP'), ('a', 'DT'), ('boy', 'NN'), ('You', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('girl', 'NN')]
['boy', 'girl']


In [ ]:
=>원 핫 인코딩: 범주형 데이터의 경우 각 범주를 하나의 열로 만들고
    자신의 범주에 해당하는 열에만 1을 설정하고 나머지는 0으로 설정하는 것
    자연어 처리에서 유사도 측정을 하기 위해서는 각 단어를 숫자로 변환을 해서 
    거리를 계산을 해야 하기 때문에 이 작업을 합니다.


=>영문 형태소 분석의 정확도가 떨어지는 경우 - 자연어는 시간이 흐르면 새로운 단어가 만들어지고 이전의 단어들이 없어기도 합니다.
많은 텍스트 문서를 모아서 모델을 계속해서 훈련을 시키는 것이 가장 좋습니다.

현실적으로 어려운 문제라서 계속해서 업데이트가 되는 brown corpus 를 활용하거나 Backoff n-gram 모델을 이용해서 다른 단어의 품사를 예측해서 이용

브라운 코퍼스에는 카테고리 별로 아주 많은 문장들이 만들어져 있습니다.
최근에는 google 의 bert 가 훨씬 더 문장을 소유하고 있음

In [16]:
#품사 태깅 후 원 핫 인코딩
maxims = ["The harder you work the more likely you can reach the goal",
         "Believe in yourself",
         "No pain No gain",
         "No sweat No sweet",
         "Courage is very important when it comes to anything"]

#품사 태깅한 내용을 저장할 list
tagged_maxims = []
for maxim in maxims:
    maxim_tag = nltk.pos_tag(word_tokenize(maxim))
    tagged_maxims.append([tag for word, tag in maxim_tag])
    
#품사 확인
for temp in tagged_maxims:
    print(temp)
print()

#여러 개의 열에 1을 설정할 수 있는 One Hot Encoding
from sklearn.preprocessing import MultiLabelBinarizer

#각 문장이 어떤 품사들로 구성되어 있는지 확인 가능
one_hot_multi = MultiLabelBinarizer()
print(one_hot_multi.fit_transform(tagged_maxims))

['DT', 'NN', 'PRP', 'VBP', 'DT', 'RBR', 'JJ', 'PRP', 'MD', 'VB', 'DT', 'NN']
['NNP', 'IN', 'PRP']
['DT', 'NN', 'RB', 'NN']
['DT', 'NN', 'NNP', 'NN']
['NN', 'VBZ', 'RB', 'JJ', 'WRB', 'PRP', 'VBZ', 'TO', 'NN']

[[1 0 1 1 1 0 1 0 1 0 1 1 0 0]
 [0 1 0 0 0 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 1 1 0 1 0 0 1 1]]


=>영문 형태소 분석의 정확도가 떨어지는 경우 - 자연어는 시간이 흐르면 새로운 단어가 만들어지고 이전의 단어들이 없어기도 합니다.
많은 텍스트 문서를 모아서 모델을 계속해서 훈련을 시키는 것이 가장 좋습니다.

현실적으로 어려운 문제라서 계속해서 업데이트가 되는 brown corpus 를 활용하거나 Backoff n-gram 모델을 이용해서 다른 단어의 품사를 예측해서 이용

브라운 코퍼스에는 카테고리 별로 아주 많은 문장들이 만들어져 있습니다.
최근에는 google 의 bert 가 훨씬 더 문장을 소유하고 있음

BoW(Bag of Word) - 단어의 등장 횟수
=>텍스트 데이터에서 특정 단어의 등장 횟수를 나타내는 특성을 만드는 작업
단어의 중요도를 파악하기 위해서 필요한 작업
=>sklearn 의 CountVectorizer 객체를 이용하면 BoW 행렬을 생성할 수 있습니다.
=>객체를 생성할 때 ngram_range 옵션을 이용해서 ngram 지정 가능
=>stop_words 를 이용해서 불용어 제거 가능
=>vocabulary 옵션을 이용해서 특정 단어의 개수만 세는 것도 가능
=>max_df 에 단어가 등장할 문서의 최대 개수를 설정
=>min_df 에 단어가 등장할 문서의 최소 개수를 설정
=>max_feature 속성을 이용해서 상위 몇개의 단어만 추출

=>get_feature_names 속성을 이용하면 각 열에 연결된 단어를 확인할 수 있습니다.

In [18]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

#테스트 데이터 만들기
text_data = np.array([
    'I love newziland. newziland', 
    'Sweden is best',
    'Germany beats both'
])

#BoW 특성 행렬 생성
count = CountVectorizer()
#희소행렬을 생성
bag_of_words = count.fit_transform(text_data)
print(bag_of_words)
print()

#밀집행렬을 출력
print(bag_of_words.toarray())
print()

#각 열의 이름을 확인
print(count.get_feature_names())
print()

  (0, 5)	1
  (0, 6)	2
  (1, 7)	1
  (1, 4)	1
  (1, 1)	1
  (2, 3)	1
  (2, 0)	1
  (2, 2)	1

[[0 0 0 0 0 1 2 0]
 [0 1 0 0 1 0 0 1]
 [1 0 1 1 0 0 0 0]]

['beats', 'best', 'both', 'germany', 'is', 'love', 'newziland', 'sweden']



In [ ]:
단어의 가중치 부여
=>tf-idf(term frequency - inverse document frequency)
tf: 하나의 단어가 하나의 문서에서 여러 번 등장하면 중요도가 높다.
idf: 하나의 단어가 여러 문서에서 여러 번 등장하면 중요도가 낮다.
=>sklearn 의 TfidfVectorizer 클래스가 계산을 수행해 줌
=>계산 방식
log((1+문서의개수)/1 + 단어t 의 문서 빈도) + 1

In [20]:
#tf-idf : 단어의 가중치 부여
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

#테스트 데이터 만들기
text_data = np.array([
    'I love newziland. newziland', 
    'Sweden is best',
    'Germany beats both',
    'Korea newziland is top of the top top'
])

#단어의 중요도를 희소행렬로 출력
#하나의 문장에서 자주 등장하면 중요도는 높아짐
#여러 문장에서 자주 등장하면 중요도는 낮아짐
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(text_data)
print(feature_matrix)
print()

#특성의 이름 파악 
print(tfidf.vocabulary_)
print()

#밀집 행렬로 변환
print(feature_matrix.toarray())
print()

  (0, 7)	0.8444932017012523
  (0, 6)	0.5355662725381126
  (1, 1)	0.6176143709756019
  (1, 4)	0.48693426407352264
  (1, 9)	0.6176143709756019
  (2, 2)	0.5773502691896257
  (2, 0)	0.5773502691896257
  (2, 3)	0.5773502691896257
  (3, 10)	0.2747918015856176
  (3, 8)	0.2747918015856176
  (3, 11)	0.8243754047568529
  (3, 5)	0.2747918015856176
  (3, 4)	0.21664901266330147
  (3, 7)	0.21664901266330147

{'love': 6, 'newziland': 7, 'sweden': 9, 'is': 4, 'best': 1, 'germany': 3, 'beats': 0, 'both': 2, 'korea': 5, 'top': 11, 'of': 8, 'the': 10}

[[0.         0.         0.         0.         0.         0.
  0.53556627 0.8444932  0.         0.         0.         0.        ]
 [0.         0.61761437 0.         0.         0.48693426 0.
  0.         0.         0.         0.61761437 0.         0.        ]
 [0.57735027 0.         0.57735027 0.57735027 0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.21664901 0.2747918
  0.  

시계열 데이터

1.시계열 데이터
=>날짜 와 시간을 이용해서 정렬된 데이터
1)제공되는 사용할 수 있는 데이터 셋
=> UCI 머신러닝 저장소
  - https://perma.cc/56Q5-YPNT
  - https://perma.cc/U6MU-2SCZ : 요가 수행자의 성별에 따른 요가 동작의 변환 데이터
  - https://perma.cc/Y34R-UGMD :  와인 데이터 셋으로 시간 값은 없지만 일정한 패턴을 갖는 형태의 데이터로 시계열 분석에서 사용할 수 있음

=>정부 시계열 데이터 셋

=>CompEngine - https://arxiv.org/abs/1905.01042

2)발견된 시계열 데이터
=>타임스탬프가 찍힌 이벤트 기록 - 항해 일지, 항공 운항 기록 등
=>타임스탬프는 없지만 일정한 간격을 갖는 데이터 - 동영상에서의 이미지
=>물리적 흔적: 의학, 청각학, 기상학 등에서의 

3)시계열 자료형
=>pandas 에서는 datetime64(TimeStamp) 과 Period(간격) 자료형 사용
=>문자열을 datetime64로 변환: pandas.to_datetime
  - format 매개변수에 날짜 서식을 설정
  - errors 매개변수에 ignore를 입력하면 에러를 무시하고 원본 문자열을 리턴하고 coerce를 에러가 발생하면 멈추지는 않지만 에러가 난 값을 NaN(NaT) 으로 설정하고 raise 로 설정하면 에러가 발생하면 예외를 발생시킴
=>날짜 와 시간 포맷
  %Y: 연도
  %m: 월
  %d: 일
  %l: 시간
  %p: AM 또는 PM
  %M: 분
  %S: 초

=>변환

문자열을 시계열로 변환

In [6]:
import pandas as pd

stock = pd.read_csv('c://programming/python/pandasdata/stock-data.csv')

#첫번째 열인 Date의 자료형이 object
stock.head()
stock.info()
print()

stock['NewDate'] = pd.to_datetime(stock['Date'])
stock.info()
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     20 non-null     object        
 1   Close    20 non-null     int64         
 2   Start    20 non-null     int64         
 3   High     20 non-null     int64         
 4   Low      20 non-null     int64         
 5   Volume   20 non-null     int64         
 6   NewDate  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memor